In [16]:
from prl_structure import PRLStructure
from structure_tools import sort_x_by_y, get_ele_list_from_struct, canonicalize_config, get_density_from_pt, substitute_configuration, scale_struct, gen_replacement_dict, substitute_configuration_with_metadata
from endmember_tools import get_sublattice_information, get_templates, get_endmembers_with_templates
from dilute_structure_tools import dilute_substitution
from database_tools import get_structures_from_database
from sqs_tools import AbstractSQS, enumerate_sqs
from pymatgen.core import Structure


## Generating the ESPEI JSON from Pysipfenn 

### 1. Endmembers generation 

#### Load the POSCAR template of target phase

In [17]:
POSCAR_name = 'Al3Ni2.poscar'
str_name = 'Al3Ni2'

with open(POSCAR_name, 'r', encoding='utf-8') as file:
    POSCAR_STR = file.read()
structure = Structure.from_str(POSCAR_STR, fmt='POSCAR')

In [18]:
structure

Structure Summary
Lattice
    abc : 3.994195728785262 3.994195728785262 4.880862
 angles : 90.0 90.0 120.00000000000001
 volume : 67.435075217798
      A : 1.997097864392631 -3.4590749688153366 0.0
      B : 1.997097864392631 3.4590749688153366 0.0
      C : 0.0 0.0 4.880862
    pbc : True True True
PeriodicSite: Al (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Al (1.9971, 1.1530, 3.1632) [0.3333, 0.6667, 0.6481]
PeriodicSite: Al (1.9971, -1.1530, 1.7177) [0.6667, 0.3333, 0.3519]
PeriodicSite: Ni (1.9971, 1.1530, 0.7154) [0.3333, 0.6667, 0.1466]
PeriodicSite: Ni (1.9971, -1.1530, 4.1655) [0.6667, 0.3333, 0.8534]

In [21]:
try:
    prls = PRLStructure.from_structure(structure)
    wyckoff_site_list, sublattice_name, sublattice_site_ratio = get_sublattice_information(structure, use_equivalent_atom=True)
    template_structure, template_configuration = get_templates(structure, wyckoff_site_list, sublattice_name, equivalent_sites=None)
    comb, endmembers = get_endmembers_with_templates(template_structure, template_configuration, sublattice_configuration)
except Exception as e:
    print(f"An error occurred: {e}")


Wyckoff sites may not be consistent with equivalent atoms in the structure, please check symmetry information and choose the sublattice model.
An error occurred: name 'substitute_configuration' is not defined


In [19]:
#Analyze sublattice inforamtion, please be careful about equivalent atoms with Wyckoff sites analyzed by pymatgen
prls=PRLStructure.from_structure(structure)
print('sublattice_site_ratios', prls.sublattice_site_ratios)
print('wyckoff_sites', prls.wyckoff_sites)
#wyckoff_site_list, sublattice_name=get_sublattice_information(structure, use_equivalent_atom=False)
wyckoff_site_list, sublattice_name, sublattice_site_ratio=get_sublattice_information(structure, use_equivalent_atom=True)
template_structure, template_configuration=get_templates(structure, wyckoff_site_list, sublattice_name, equivalent_sites=None)
sublattice_configuration=[['Al','Fe','Ni'],['Al','Fe','Ni'],['Al','Fe','Ni']]
#sublattice_configuration=[['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Fe','Ni'],['Fe','Ni'],['Fe','Ni'],['Fe','Ni'],['Al'],['Al'],['Al'],['Al'],['Al'],['Fe','Ni']]
comb, endmembers=get_endmembers_with_templates(template_structure, template_configuration, sublattice_configuration)

sublattice_site_ratios [1, 2, 2]
wyckoff_sites ['a', 'd', 'd2']
Wyckoff sites may not be consistent with equivalent atoms in the structure, please check symmetry information and choose the sublattice model.


NameError: name 'substitute_configuration' is not defined

### Generate and save the target phase endmembers into the directory 

In [28]:
import os

# Create the directory if it doesn't exist
directory_name = str_name+"_endmembers"
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

for i in range(0, len(endmembers)):
    filename = os.path.join(directory_name, str(i+1) +'_'+directory_name +'.POSCAR')
    endmembers[i].sort()
    endmembers[i].to(fmt='poscar', filename=filename)
    firstline = 'POSCAR' + ' ' + 'sublattice information' + ' ' + str(sublattice_name) + str(sublattice_site_ratio) + str(comb[i]) + ' ' + 'generate using OcMAT structure builders\n'
    #print(str(comb[i]))
    with open(filename, 'r', encoding='utf-8') as file:
        data = file.readlines()
        data[0] = firstline
        
    with open(filename, 'w', encoding='utf-8') as file:
        file.writelines(data)


#### Generate dilute structures base on endmember structure list

In [23]:
with open(POSCAR_name, 'r', encoding='utf-8') as file:
    POSCAR_STR = file.read()
structure = Structure.from_str(POSCAR_STR, fmt='POSCAR')
prls=PRLStructure.from_structure(structure)
print('sublattice_site_ratios', prls.sublattice_site_ratios)
print('wyckoff_sites', prls.wyckoff_sites)
wyckoff_site_list, sublattice_name, sublattice_site_ratio=get_sublattice_information(structure, use_equivalent_atom=True)
template_structure, template_configuration=get_templates(structure, wyckoff_site_list, sublattice_name, equivalent_sites=None)
sublattice_configuration=[['Al','Fe','Ni'],['Al','Fe','Ni'],['Al','Fe','Ni']]
#sublattice_configuration=[['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Fe','Ni'],['Fe','Ni'],['Fe','Ni'],['Fe','Ni'],['Al'],['Al'],['Al'],['Al'],['Al'],['Fe','Ni']]
comb, endmembers=get_endmembers_with_templates(template_structure, template_configuration, sublattice_configuration)

sublattice_site_ratios [1, 2, 2]
wyckoff_sites ['a', 'd', 'd']
Wyckoff sites may not be consistent with equivalent atoms in the structure, please check symmerty information and choose the sublattice model.


In [29]:
#dilute, sublattice_conf=dilute_substitution(endmembers, {'d': ['Al'], 'g': ['Al'],'i': ['Al'],'i2': ['Al'],'i3': ['Al'],'i4': ['Al'],'i5': ['Al'],'i6': ['Al'],'i7': ['Al'],'i8': ['Al'],'i9': ['Fe','Ni'],'i10': ['Fe','Ni'],'i11': ['Fe','Ni'],'i12': ['Fe','Ni'],'j': ['Al'], 'j2': ['Al'],'j3': ['Al'],'j4': ['Al'],'j5': ['Al'],'j6': ['Fe','Ni']})
dilute, sublattice_conf=dilute_substitution(endmembers, {'a': ['Al','Fe','Ni'], 'd': ['Al','Fe','Ni'],'d2': ['Al','Fe','Ni']})

In [30]:
import os

# Create the directory if it doesn't exist
directory_name = str_name+"_dilute"
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

for i in range(0, len(dilute)):
    filename = os.path.join(directory_name, str(i+1) +'_'+directory_name +'.POSCAR')
    dilute[i].sort()
    dilute[i].to(fmt='poscar', filename=filename)
    firstline = 'POSCAR' + ' ' + 'sublattice information' +str(sublattice_conf[i]) + 'generate using OcMAT structure builders\n'
    
    with open(filename, 'r', encoding='utf-8') as file:
        data = file.readlines()
        data[0] = firstline
        
    with open(filename, 'w', encoding='utf-8') as file:
        file.writelines(data)

# SQS generation

#### For ternary BCC SQS structures, you can get the templates from PRL website: https://phaseslab.com/#/resources/
#### For Ternary HCP SQS structures, you can generature structures using ATAT SQS database

### Get FCC_A1 SQS from SQS ATAT database

In [102]:
from tinydb import TinyDB
dbjson=TinyDB('ATATSQS/ATAT_SQSDB.json')
prototype_name='FCC_A1'
system = 'Al,Fe,Ni'
subl_model=[['Al','Fe','Ni']]
subl_site_ratios=[1]

In [103]:
absqs=get_structures_from_database(dbjson, prototype_name, subl_model, subl_site_ratios)
concrete_structure={}
absqs_pymatgen={}
for j in range(0,len(absqs)):
    absqs_pymatgen[j]=AbstractSQS.from_dict(absqs[j])
    concrete_structure[j] = enumerate_sqs(absqs_pymatgen[j], subl_model)
print(absqs_pymatgen)

{0: Structure Summary
Lattice
    abc : 1.224744871391589 2.1213203435596424 3.4641016151377544
 angles : 90.0 90.0 90.0
 volume : 9.0
      A : 0.5 -0.5 -1.0
      B : -1.5 -1.5 0.0
      C : -2.0 2.0 -2.0
    pbc : True True True
PeriodicSite: Xaa0+ (-1.5000, -1.0000, -0.5000) [0.1667, 0.8333, 0.1667]
PeriodicSite: Xaa0+ (-3.0000, 0.5000, -2.5000) [0.5000, 0.8333, 1.0000]
PeriodicSite: Xaa0+ (-2.5000, 0.5000, -3.0000) [1.0000, 0.6667, 1.0000]
PeriodicSite: Xac0+ (-2.5000, 0.0000, -1.5000) [0.1667, 0.8333, 0.6667]
PeriodicSite: Xac0+ (-2.0000, -0.5000, -1.5000) [0.5000, 0.8333, 0.5000]
PeriodicSite: Xaa0+ (-1.5000, -0.5000, -2.0000) [1.0000, 0.6667, 0.5000]
PeriodicSite: Xab0+ (-1.5000, -1.0000, -1.5000) [0.8333, 0.8333, 0.3333]
PeriodicSite: Xab0+ (-1.5000, -0.5000, -1.0000) [0.3333, 0.6667, 0.3333]
PeriodicSite: Xaa0+ (-1.0000, -1.0000, -1.0000) [0.6667, 0.6667, 0.1667]
PeriodicSite: Xab0+ (-1.0000, -0.5000, -0.5000) [0.1667, 0.5000, 0.1667]
PeriodicSite: Xab0+ (-2.5000, 1.0000, -2.

In [101]:
a=6
for i in range(0, len(concrete_structure[a])):
    #print(i)
    #print(type(concrete_structure))
    print(concrete_structure[a][0]) #tenary hcp sqs: 1, 4 ; 1,7; 1,11; 5 4;

Full Formula (Al32)
Reduced Formula: Al
abc   :   6.401877   9.495519   9.917746
angles:  64.202352  82.582444  78.330854
pbc   :       True       True       True
Sites (32)
  #  SP         a       b        c
---  ----  ------  ------  -------
  0  Al    1       1       1
  1  Al    0.5625  0.0625  0.09375
  2  Al    0.125   0.125   0.1875
  3  Al    1       1       0.5
  4  Al    0.6875  0.1875  0.28125
  5  Al    0.375   0.375   0.0625
  6  Al    0.5625  0.0625  0.59375
  7  Al    0.25    0.25    0.375
  8  Al    0.125   0.125   0.6875
  9  Al    0.9375  0.4375  0.15625
 10  Al    0.8125  0.3125  0.46875
 11  Al    0.5     0.5     0.25
 12  Al    0.1875  0.6875  0.03125
 13  Al    0.6875  0.1875  0.78125
 14  Al    0.375   0.375   0.5625
 15  Al    0.0625  0.5625  0.34375
 16  Al    0.25    0.25    0.875
 17  Al    0.75    0.75    0.125
 18  Al    0.9375  0.4375  0.65625
 19  Al    0.625   0.625   0.4375
 20  Al    0.3125  0.8125  0.21875
 21  Al    0.8125  0.3125  0.96875
 22  Al   

In [105]:
import os
from pymatgen.core import Structure

# List of cases to go through
cases = [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0,5), (0,6), (0,7),(0,8),(0,9),
         (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1,5), (1,6), (1,7),(1,8),
         (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2,5), (2,6), (2,7),(2,8),
         (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3,5), (3,6), (3,7),(3,8),
         (4, 0), (4, 1), (4, 2),
         (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5,5),
         (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6,5),(6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6,11),(6, 12), (6, 13), (6,14)]

# Directory to save POSCAR files
directory_name = "FCC_SQS"
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

# Iterate over each case
for case in cases:
    # Extract structure from concrete_structure
    structure_dict = concrete_structure[case[0]][case[1]].as_dict()
    structure = Structure.from_dict(structure_dict)

    # Convert to POSCAR format string and then create sorted Structure object
    poscar_str = structure.to(fmt='poscar')
    poscar_sorted = Structure.from_str(poscar_str, fmt='POSCAR', sort=True)

    # Create filename for the POSCAR file
    filename = os.path.join(directory_name, f"FCCSQS_{case[0]}_{case[1]}.POSCAR")

    # Save the sorted structure to a file in POSCAR format
    poscar_sorted.to(filename=filename, fmt='poscar')

    # Optionally, print the structure or filename
    print(f"Saved POSCAR file for case {case} as {filename}")


Saved POSCAR file for case (0, 0) as FCC_SQS/FCCSQS_0_0.POSCAR
Saved POSCAR file for case (0, 1) as FCC_SQS/FCCSQS_0_1.POSCAR
Saved POSCAR file for case (0, 2) as FCC_SQS/FCCSQS_0_2.POSCAR
Saved POSCAR file for case (0, 3) as FCC_SQS/FCCSQS_0_3.POSCAR
Saved POSCAR file for case (0, 4) as FCC_SQS/FCCSQS_0_4.POSCAR
Saved POSCAR file for case (0, 5) as FCC_SQS/FCCSQS_0_5.POSCAR
Saved POSCAR file for case (0, 6) as FCC_SQS/FCCSQS_0_6.POSCAR
Saved POSCAR file for case (0, 7) as FCC_SQS/FCCSQS_0_7.POSCAR
Saved POSCAR file for case (0, 8) as FCC_SQS/FCCSQS_0_8.POSCAR
Saved POSCAR file for case (0, 9) as FCC_SQS/FCCSQS_0_9.POSCAR
Saved POSCAR file for case (1, 0) as FCC_SQS/FCCSQS_1_0.POSCAR
Saved POSCAR file for case (1, 1) as FCC_SQS/FCCSQS_1_1.POSCAR
Saved POSCAR file for case (1, 2) as FCC_SQS/FCCSQS_1_2.POSCAR
Saved POSCAR file for case (1, 3) as FCC_SQS/FCCSQS_1_3.POSCAR
Saved POSCAR file for case (1, 4) as FCC_SQS/FCCSQS_1_4.POSCAR
Saved POSCAR file for case (1, 5) as FCC_SQS/FCCSQS_1_5

#### Try to generate the SQS BCC from ATAT database

In [106]:
from tinydb import TinyDB
dbjson=TinyDB('ATATSQS/ATAT_SQSDB.json')
prototype_name='BCC_A2'
system = 'Al,Fe,Ni'
subl_model=[['Al','Fe','Ni']]
subl_site_ratios=[1]


In [107]:
absqs=get_structures_from_database(dbjson, prototype_name, subl_model, subl_site_ratios)
concrete_structure={}
absqs_pymatgen={}
for j in range(0,len(absqs)):
    absqs_pymatgen[j]=AbstractSQS.from_dict(absqs[j])
    concrete_structure[j] = enumerate_sqs(absqs_pymatgen[j], subl_model)
print(absqs_pymatgen)

{0: Structure Summary
Lattice
    abc : 3.1622776601683795 3.1622776601683795 3.3166247903554
 angles : 112.41950137364931 112.41950137364931 53.130102354155994
 volume : 24.0
      A : 0.0 -1.0 3.0
      B : 0.0 -3.0 1.0
      C : 3.0 1.0 -1.0
    pbc : True True True
PeriodicSite: Xaa0+ (3.0000, -1.0000, 2.0000) [0.8750, 0.3750, 1.0000]
PeriodicSite: Xaa0+ (3.0000, -2.0000, 2.0000) [0.7500, 0.7500, 1.0000]
PeriodicSite: Xab0+ (0.5000, -3.5000, 3.5000) [0.9167, 0.9167, 0.1667]
PeriodicSite: Xab0+ (3.0000, 0.0000, 1.0000) [0.6250, 0.1250, 1.0000]
PeriodicSite: Xaa0+ (3.0000, -1.0000, 1.0000) [0.5000, 0.5000, 1.0000]
PeriodicSite: Xac0+ (0.5000, -1.5000, 2.5000) [0.7917, 0.2917, 0.1667]
PeriodicSite: Xac0+ (3.0000, -2.0000, 1.0000) [0.3750, 0.8750, 1.0000]
PeriodicSite: Xaa0+ (0.5000, -2.5000, 2.5000) [0.6667, 0.6667, 0.1667]
PeriodicSite: Xab0+ (1.0000, -2.0000, 3.0000) [0.9583, 0.4583, 0.3333]
PeriodicSite: Xaa0+ (1.0000, -3.0000, 3.0000) [0.8333, 0.8333, 0.3333]
PeriodicSite: Xac0+ (

In [109]:
a=6
for i in range(0, len(concrete_structure[a])):
    #print(i)
    #print(type(concrete_structure))
    print(concrete_structure[a][14]) #0 4;6 4 ;6 7; 6 11

Full Formula (Ni32)
Reduced Formula: Ni
abc   :   7.911176   7.911176   6.851279
angles: 106.778655 106.778655  60.000000
pbc   :       True       True       True
Sites (32)
  #  SP         a       b      c
---  ----  ------  ------  -----
  0  Ni    0.125   0.125   0.75
  1  Ni    0.3125  0.3125  0.875
  2  Ni    0.125   0.625   0.75
  3  Ni    0.3125  0.8125  0.875
  4  Ni    0.0625  0.0625  0.375
  5  Ni    0.25    0.25    0.5
  6  Ni    0.0625  0.5625  0.375
  7  Ni    0.625   0.125   0.75
  8  Ni    0.4375  0.4375  0.625
  9  Ni    0.25    0.75    0.5
 10  Ni    0.8125  0.3125  0.875
 11  Ni    0.625   0.625   0.75
 12  Ni    0.4375  0.9375  0.625
 13  Ni    0.8125  0.8125  0.875
 14  Ni    1       1       1
 15  Ni    0.1875  0.1875  0.125
 16  Ni    1       0.5     1
 17  Ni    0.5625  0.0625  0.375
 18  Ni    0.375   0.375   0.25
 19  Ni    0.1875  0.6875  0.125
 20  Ni    0.75    0.25    0.5
 21  Ni    0.5625  0.5625  0.375
 22  Ni    0.375   0.875   0.25
 23  Ni    0.9375  0.

In [111]:
import os
from pymatgen.core import Structure

# List of cases to go through
cases = [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0,5), (0,6), (0,7),(0,8),(0,9),
         (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1,5), (1,6), (1,7),(1,8),
         (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2,5), (2,6), (2,7),(2,8),
         (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3,5), (3,6), (3,7),(3,8),
         (4, 0), (4, 1), (4, 2),
         (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5,5),
         (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6,5),(6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6,11),(6, 12), (6, 13), (6,14)]

# Directory to save POSCAR files
directory_name = "BCC_SQS"
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

# Iterate over each case
for case in cases:
    # Extract structure from concrete_structure
    structure_dict = concrete_structure[case[0]][case[1]].as_dict()
    structure = Structure.from_dict(structure_dict)

    # Convert to POSCAR format string and then create sorted Structure object
    poscar_str = structure.to(fmt='poscar')
    poscar_sorted = Structure.from_str(poscar_str, fmt='POSCAR', sort=True)

    # Create filename for the POSCAR file
    filename = os.path.join(directory_name, f"BCCSQS_{case[0]}_{case[1]}.POSCAR")

    # Save the sorted structure to a file in POSCAR format
    poscar_sorted.to(filename=filename, fmt='poscar')

    # Optionally, print the structure or filename
    print(f"Saved POSCAR file for case {case} as {filename}")

Saved POSCAR file for case (0, 0) as BCC_SQS/BCCSQS_0_0.POSCAR
Saved POSCAR file for case (0, 1) as BCC_SQS/BCCSQS_0_1.POSCAR
Saved POSCAR file for case (0, 2) as BCC_SQS/BCCSQS_0_2.POSCAR
Saved POSCAR file for case (0, 3) as BCC_SQS/BCCSQS_0_3.POSCAR
Saved POSCAR file for case (0, 4) as BCC_SQS/BCCSQS_0_4.POSCAR
Saved POSCAR file for case (0, 5) as BCC_SQS/BCCSQS_0_5.POSCAR
Saved POSCAR file for case (0, 6) as BCC_SQS/BCCSQS_0_6.POSCAR
Saved POSCAR file for case (0, 7) as BCC_SQS/BCCSQS_0_7.POSCAR
Saved POSCAR file for case (0, 8) as BCC_SQS/BCCSQS_0_8.POSCAR
Saved POSCAR file for case (0, 9) as BCC_SQS/BCCSQS_0_9.POSCAR
Saved POSCAR file for case (1, 0) as BCC_SQS/BCCSQS_1_0.POSCAR
Saved POSCAR file for case (1, 1) as BCC_SQS/BCCSQS_1_1.POSCAR
Saved POSCAR file for case (1, 2) as BCC_SQS/BCCSQS_1_2.POSCAR
Saved POSCAR file for case (1, 3) as BCC_SQS/BCCSQS_1_3.POSCAR
Saved POSCAR file for case (1, 4) as BCC_SQS/BCCSQS_1_4.POSCAR
Saved POSCAR file for case (1, 5) as BCC_SQS/BCCSQS_1_5

In [164]:
Ternary_system = ['HF','MO','TI']
phase_name = "HCP_A3"
sublattice_site_ratios = [1,0.5]
components= ['HF','MO','TI', 'VA']

In [165]:
# Replace 'your_file.csv' with the actual path to your CSV file
csv_file = 'SQS_HCP_pySIPFENN.csv'
df = pd.read_csv(csv_file) # dft
#df.head()
# Constant for conversion from eV/atom to J/mol-atom
conversion_factor = 96491

# Adding the new column
df["HM_FORM(J/mol-atom)"] = df["SIPFENN_Krajewski2022_NN30"] * conversion_factor
# Creating a list and appending values from the HM_FORM(J/mol-atom) column
HM_FORM = df["HM_FORM(J/mol-atom)"].tolist()
HM_FORM

[23974.695418700576, 43116.143703728914, 26273.093582779165, 27269.1977776587]

In [166]:
configuration_1 = [['HF','MO','TI'],'VA']
configuration_2 = [['HF','MO','TI'],'VA']
configuration_3 = [['HF','MO','TI'],'VA']
configuration_4 = [['HF','MO','TI'],'VA']
configuration = [configuration_1,configuration_2,configuration_3,configuration_4]

sublattice_occupancies_1 = [[0.5,0.25,0.25],1]
sublattice_occupancies_2 = [[0.25,0.5,0.25],1]
sublattice_occupancies_3 = [[0.25,0.25,0.5],1]
sublattice_occupancies_4 = [[0.333,0.333,0.334],1]
sublattice_occupancies = [sublattice_occupancies_1,sublattice_occupancies_2,sublattice_occupancies_3,sublattice_occupancies_4]

In [168]:
# Define the elements and their state
elements = ['HF', 'MO', 'TI']
state = 'VA'

# Create unique configurations by combining elements with the state
configuration_1 = [elements, state]
configuration_2 = [elements, state]
configuration_3 = [elements, state]
configuration_4 = [elements, state]
configurations = [configuration_1, configuration_2, configuration_3, configuration_4]

# Define the sublattice occupancies
sublattice_occupancies_1 = [[0.5, 0.25, 0.25], 1]
sublattice_occupancies_2 = [[0.25, 0.5, 0.25], 1]
sublattice_occupancies_3 = [[0.25, 0.25, 0.5], 1]
sublattice_occupancies_4 = [[0.333, 0.333, 0.334], 1]
sublattice_occupancies = [sublattice_occupancies_1, sublattice_occupancies_2, sublattice_occupancies_3, sublattice_occupancies_4]

# Display the configurations and sublattice occupancies
print("Configurations:", configurations)
print("Sublattice Occupancies:", sublattice_occupancies)



Configurations: [[['HF', 'MO', 'TI'], 'VA'], [['HF', 'MO', 'TI'], 'VA'], [['HF', 'MO', 'TI'], 'VA'], [['HF', 'MO', 'TI'], 'VA']]
Sublattice Occupancies: [[[0.5, 0.25, 0.25], 1], [[0.25, 0.5, 0.25], 1], [[0.25, 0.25, 0.5], 1], [[0.333, 0.333, 0.334], 1]]


In [169]:
dictionary1 ={
  "components": components,
  "phases": [phase_name],
  "solver": {
	  "mode": "manual",
	  "sublattice_site_ratios": sublattice_site_ratios,
      "sublattice_occupancies": sublattice_occupancies,
	  "sublattice_configurations": configuration
  },
  "conditions": {
	  "P": 101325,
	  "T": 298.15
  },
  "output": "HM_FORM",
    "values":   [[HM_FORM]],
  "reference": "SQS_SIPFEEN"
}
with open(phase_name+"_HMFORM_sqs_structure_SIPFENN.json", "w") as outfile:
        json.dump(dictionary1, outfile,indent=1)